# Cleaning, saving, erasing

In [ ]:
from reader import Reader
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from datetime import datetime
from datetime import date as date_creator
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from numba import njit
import pandas as pd
import os
from numba import jit

%matplotlib inline

In [ ]:
def read_igc(file):
    with open(file, 'r') as f:
        parsed_igc_file = Reader().read(f)
    # print('igc_file created')
    previous_lat = 0
    previous_lon = 0
    
    all_lon = np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_lat = np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_speed=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_vz=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_alti=np.zeros(len(parsed_igc_file['fix_records'][1]))
    all_time=[0 for _ in range(len(parsed_igc_file['fix_records'][1]))]

    for i,record in enumerate(parsed_igc_file['fix_records'][1]):
        record['time'] = record['time'].replace(hour=record['time'].hour + 1)
        all_lat[i] = record['lat']
        all_lon[i] = record['lon']
        if previous_lon ==0 :
            previous_lat = record['lat']
            previous_lon = record['lon']
            previous_datetime = record['time']
            previous_alt_gps = record['gps_alt']
            # previous_alt_baro = record['pressure_alt']
            all_time[i] = record['time']
        else :
            dxy = abs(compute_dist(previous_lat,previous_lon,record['lat'],record['lon'],rad=False))
            # dz = previous_alt_baro-record['pressure_alt']
            dz = previous_alt_gps-record['gps_alt']
            dt = get_date_time_dif(record['time'],previous_datetime)
            # if dt>2 : print('dtttttt>2 ',dt)
            if dt>0 :
                all_speed[i] = min(100,dxy/dt*3.6)
                all_vz[i] = dz/dt
            all_alti[i] = record['gps_alt']
            all_time[i] = record['time']
            
            previous_lat = record['lat']
            previous_lon = record['lon']
            previous_datetime = record['time']
            previous_alt_gps = record['gps_alt']
            # previous_alt_baro = record['pressure_alt']

            # if previous_datetime.hour>10 :
            #     all_speed = all_speed[:i]
            #     all_vz = all_vz[:i]
            #     all_alti = all_alti[:i]
            #     break
#    return all_lon,all_lat,all_speed,all_vz,remove_zero_from_alti(all_alti),all_time
    return all_lon,all_lat,all_alti,all_time

# Parapente traces to clean : Done

In [ ]:
def read_igc_2(file):
    try:
        with open(file, 'r') as f:
            parsed_igc_file = Reader().read(f)
    except ValueError:
        print(file)
        os.remove(file)

path_cleanning = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/Parapente/'
list_files_cleanning = [f for f in listdir(path_cleanning) if isfile(join(path_cleanning, f))]

for file in list_files_cleanning :
    read_igc_2(join(path_cleanning,file))

In [ ]:
path_cleannings = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/'
path_para = 'Parapente/'
path_planneur = 'Planeur/'
days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']

list_files_cleannings_para =  []
list_files_cleannings_planneur = []
for i in range(len(days)):
    paths_para = join(path_cleannings,join(days[i],path_para))
#    paths_planneur = join(path_cleannings,join(days[i],path_planneur))
    list_files_cleannings_para.append([join(paths_para,f) for f in listdir(paths_para) if isfile(join(paths_para, f))])
#    list_files_cleannings_planneur.append([join(paths_planneur,f) for f in listdir(paths_planneur) if isfile(join(paths_planneur, f))])

for i in list_files_cleannings_para :
    for j in i :
        read_igc_2(j)
#for i in list_files_cleannings_planneur:
#    for j in i :
#        read_igc_2(j)

## Search for the dates and organize the parapente new data in day to day folder : Done

In [ ]:
import shutil

def move_and_delete_file(repository, filename, destination_folder):
    file_directory = join(repository, filename)
    destination_path = join(destination_folder, filename)
    shutil.copy2(file_directory, destination_path)
    os.remove(file_directory)

def find_date_in_igc(igc_file_path):
    with open(igc_file_path, 'r') as file:
        for line in file :
            day = 0
            if line.startswith('HFDTEDATE:'):
                date_str = line[10:16]
                day = date_str[:2]
                return day
            elif line.startswith('HFDTE'):
                date_str = line[4:]
                day = date_str[:2]
                return day

def find_date_in_igc_last(igc_file_path):
    with open(igc_file_path, 'r') as file:
        for line in file :
            day = 0
            if line.startswith('HFDTE'):
                date_str = line[5:]
                day = date_str[:2]
                return day


In [ ]:
path_parapente_new = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/Parapente/'
dest = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/'
list_files_parapente_new = [f for f in listdir(path_parapente_new) if isfile(join(path_parapente_new, f))]
days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']

for i in list_files_parapente_new :
    day_i = find_date_in_igc_last(join(path_parapente_new,i))
    if day_i.isdigit() :
        day_i = int(day_i)
        if day_i == 19 :
            destination_folder = join(join(dest, days[0]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)
        if day_i == 20 :
            destination_folder = join(join(dest, days[1]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)
        if day_i == 21 :
            destination_folder = join(join(dest, days[2]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)
        if day_i == 22 :
            destination_folder = join(join(dest, days[3]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)
        if day_i == 23 :
            destination_folder = join(join(dest, days[4]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)
        if day_i == 24 :
            destination_folder = join(join(dest, days[5]),'Parapente/')
            move_and_delete_file(path_parapente_new, i, destination_folder)

## Save traces one and for all

In [ ]:
@njit()    
def ndarray_index(arr, value):
    try:
        index = np.argwhere(arr == value)[0][0]
        return index
    except IndexError:
        raise ValueError(f"{value} is not in array")
        
@njit()    
def filter_list_3(lst1, lst2, threshold_low1=None, threshold_high1=None, threshold_low2=None, threshold_high2=None):
    filtered_list1 = []
    filtered_list2 = []
    indexs = []
    for item1,item2 in zip(lst1,lst2):
        if threshold_low1 is not None and item1 < threshold_low1:
            continue
        if threshold_high1 is not None and item1 > threshold_high1:
            continue
        if threshold_low2 is not None and item2 < threshold_low2:
            continue
        if threshold_high2 is not None and item2 > threshold_high2:
            continue
        indexs.append(ndarray_index(lst1,item1))
        filtered_list1.append(item1)
        filtered_list2.append(item2)
    return filtered_list1,filtered_list2,indexs

@njit()    
def missing_numbers(lst):
    if not lst:
        return []

    max_num = max(lst)
    return [num for num in range(1, max_num) if num not in lst]

@njit()   
def remove_elements_by_indices(lst, indices):
    indices = set(indices)  # Convert to set for faster lookup
    return [elem for i, elem in enumerate(lst) if i not in indices]

def repartition_coord(all_lon,all_lat,all_alti, all_time):
    liste = []
    for i in range(len(all_lon)):
        liste.append([all_lon[i],all_lat[i],all_alti[i],all_time[i]])
    return liste

def save_traces(img_extent,type_vol) :
    directory = 'C:/Users/barroisl/Documents/Stage_Léo/Vols/'
    traces_save_path = 'T:/C2H/STAGES/LEO_BARROIS/ndarray/traces/'
    days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']
    
    for j in range(len(days)): 
        low_path = join(directory,days[j])
        if type_vol == 'para' :
            low_path = low_path+'Parapente/'
        elif type_vol == 'plan' :
            low_path = low_path+'Planeur/'
        liste_f = [f for f in os.listdir(low_path)]
        for i,file in enumerate(liste_f):
            indexs = []
            print(i,end=' ')
#            all_lon,all_lat,speed_vid,vz_vid,alti_vid,time_vid,all_vz2,all_speed_smooth, all_vz_smooth, all_alti_smooth,all_alti = prep_file(join(low_path,path),file)
            all_lon,all_lat,all_alti, all_time = read_igc(join(low_path,file))
            to_save = repartition_coord(all_lon,all_lat,all_alti,all_time)
            if type_vol == 'para' :
                np.save(join(join(traces_save_path,days[j]),'Parapente/')+file+'.npy', to_save)
            elif type_vol == 'plan' :
                np.save(join(join(traces_save_path,days[j]),'Planeur/')+file+'.npy', to_save)
        print(days[j] + ' ok')

In [ ]:
img_extent = (4.7942, 8.1545, 43.3545, 46.6707)

save_traces(img_extent,'plan')
save_traces(img_extent,'para')

## Si jamais il faut effacer toutes les traces

days = ['IGC_19-08-2023/','IGC_20-08-2023/','IGC_21-08-2023/','IGC_22-08-2023/','IGC_23-08-2023/','IGC_24-08-2023/']
type_vols = ['Parapente/','Planeur/']
for j in range(6):
    for i in type_vols :
        path = 'T:/C2H/STAGES/LEO_BARROIS/ndarray/traces/'+days[j]+i
        liste_file_remove = [join(path,f) for f in os.listdir(path)]
        for file in liste_file_remove :
            os.remove(file)